In [ ]:
import zipfile
with zipfile.ZipFile('data_final.zip') as zf:
    zf.extractall('')

In [ ]:
!python -m pip install -qU detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html
!pip install -q torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -q tensorflow==2.1.0
!pip install -qU albumentations
!pip install -q opencv-python==4.5.4.60

In [ ]:
import cv2
import random
import json
import os

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import shutil
import tqdm
from matplotlib import pyplot as plt
import numpy as np

import torch, torchvision
import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog,DatasetCatalog
from detectron2.data.datasets import register_coco_instances,load_coco_json
from detectron2.data import detection_utils as utils
from detectron2.engine import DefaultTrainer
from detectron2.engine import HookBase

import albumentations as albu
from albumentations.pytorch import ToTensorV2

import logging
logger = logging.getLogger('detectron2')
logger.setLevel(logging.CRITICAL)

In [ ]:
import json
with open('/content/data_final/train_segmentation/annotations.json') as f:
    annotations = json.load(f)

annotations_train = {}
annotations_train['categories'] = annotations['categories']
annotations_train['images'] = []

for img in annotations['images']:
    annotations_train['images'].append(img)

train_img_id = [i['id'] for i in annotations_train['images']]
annotations_train['annotations'] = []

for annot in annotations['annotations']:
    annotations_train['annotations'].append(annot)

with open('/content/data_final/train_segmentation/annotations_train.json', 'w') as outfile:
    json.dump(annotations_train, outfile)

In [ ]:
for d in ['train']:
    DatasetCatalog.register("my_dataset_"+d, lambda d=d: load_coco_json(f"/content/data_final/train_segmentation/annotations_{d}.json",
    image_root= "/content/data_final/train_segmentation/images",\
    dataset_name="my_dataset_"+d,extra_annotation_keys=['bbox_mode']))

dataset_dicts_train = DatasetCatalog.get("my_dataset_train")
train_metadata = MetadataCatalog.get("my_dataset_train")

print('Размер обучающей выборки (Картинки): {}'.format(len(dataset_dicts_train)))

Размер обучающей выборки (Картинки): 932


In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")) 
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml")
cfg.SOLVER.BASE_LR = 0.001
cfg.SOLVER.STEPS = (12000, 17000, 22000, 27000, 29000)
cfg.SOLVER.GAMMA = 0.1
cfg.SOLVER.MAX_ITER = 30000
cfg.SOLVER.CHECKPOINT_PERIOD = 2000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.SOLVER.IMS_PER_BATCH = 1


cfg.DATASETS.TRAIN = ("my_dataset_train")
cfg.DATASETS.TEST = ()
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.1
cfg.INPUT.FORMAT = 'BGR'
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

cfg.OUTPUT_DIR = './output'
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [ ]:
import detectron2.data.transforms as T
from detectron2.data import DatasetMapper, build_detection_train_loader, build_detection_test_loader

class My_Augs(T.Transform):
    def __init__(self):
        self.augs = albu.Compose([
                                      albu.RandomGamma(gamma_limit=(85, 115), p=0.2),
                                      albu.OneOf([
                                                  albu.Blur(blur_limit=7, p=0.15),
                                                  albu.MedianBlur(blur_limit=7, p=0.15)
                                      ]),
                                      albu.RGBShift(r_shift_limit=9, g_shift_limit=9, b_shift_limit=9, p=0.2),
                                      albu.JpegCompression(quality_lower=75, p=0.2)])

    def apply_image(self, img):
        img = self.augs(image=img)['image']
        return img

    def apply_coords(self, coords):
        return coords.astype(np.float32)

class CustomResize(T.Augmentation):
    def get_transform(self, image):
        rand_scale = random.uniform(0.8, 1.1)
        my_scale = 2.4
        new_width = int((image.shape[1] * rand_scale) // my_scale)
        new_height = int((image.shape[0] * rand_scale) // my_scale)
        return T.ResizeTransform(image.shape[0], image.shape[1], new_height, new_width)

class Trainer(DefaultTrainer):

    @classmethod
    def build_train_loader(cls, cfg):
        mapper = DatasetMapper(cfg, is_train=True, augmentations=[
                   T.RandomBrightness(0.7, 1.3),
                   T.RandomContrast(0.7, 1.3),
                   T.RandomSaturation(0.7, 1.3),
                   T.RandomLighting(0.9),
                   My_Augs(),
                   T.RandomFlip(prob=0.5, horizontal=True, vertical=False),
                   T.RandomFlip(prob=0.2, horizontal=False, vertical=True),
                   T.RandomRotation((-5, 5), expand=True),
                   CustomResize()])
        return build_detection_train_loader(cfg, mapper=mapper)

In [ ]:
new_trainer = Trainer(cfg)
new_trainer.resume_or_load(resume=False)
new_trainer.train()